In [16]:
import numpy as np
import pandas as pd

from os import listdir
from os.path import basename, isfile, join

from matplotlib.pyplot import imshow

In [2]:
metadata = pd.read_csv(join('data', 'metadata.csv'))

In [54]:
%reload_ext autoreload
%autoreload 2
from DataGenerator import DataGenerator

datagen = DataGenerator(metadata, 32)
datagen.n_classes

345

In [ ]:
from keras.models import Sequential, Model
from keras import Input
from keras.layers import Dense, Reshape, Flatten, Conv2D, LeakyReLU, Concatenate

def discriminator():
    dropout=0.4
    depth=64
    alpha=0.2
    
    label = Input(shape=(1,))
    x = Embedding(345,50)(label)
    x = Dense(784)(x)
    x = Reshape((28,28,1))(x)
    
    image = Input(shape=(28,28,1))
    
    concat = Concatenate()[image,x]
    x = Conv2D(1*depth,5,strides=2,padding='same')(concat)
    x = Dropout(dropout)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = Conv2D(2*depth,5,strides=2,padding='same')(x)
    x = Dropout(dropout)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = Conv2D(4*depth,5,strides=2,padding='same')(x)
    x = Dropout(dropout)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = Conv2D(8*depth,5,strides=1,padding='same')(x)
    x = Dropout(dropout)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = Flatten()(x)
    out = Dense(1,activation='sigmoid')(x)
    
    model = Model(inputs=[image,label], outputs=out)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
    
    
    
    
